In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [ ]:
(ds_train, ds_test), ds_info = tfds.load('cats_vs_dogs',split=['train[:80%]', 'train[80%:]'],with_info=True,as_supervised=True)
Number_of_hidden_layers = 6
num_px=128

In [ ]:
def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (num_px, num_px))  # Standardizing image dimensions.
    image = image / 255.0  # Normalization to the [0, 1] range.
    return image, label

# Apply the pre-processing function, batch the datasets, and prefetch for performance.
ds_train = ds_train.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
for x in ds_train.take(1):
    image, label = x
    plt.figure(figsize=(1, 1))
    plt.imshow(image.numpy()[3])  # Convert tensor to NumPy array.
    plt.title(f'Label: {label.numpy()[3]}')
    plt.axis('off')  # Suppress axis display for clarity.
    plt.show()
    label = label.numpy().reshape(-1, 1)
    image = image.numpy().reshape(image.shape[0], -1).T

In [ ]:
n_x=image.shape[0]
n_y=1
n_h=7
print(n_x)
layers_dims = [n_x, n_x//2, n_x//3, n_h, 5, n_y]
n_layers = len(layers_dims)
print(f"Number of layers: {n_layers}")

In [ ]:
def sigmoid(Z):
    cache=[]
    A=1/(1+np.exp(-Z))
    cache.append(Z)
    return A,cache

In [ ]:
def relu(Z):
    cache=[]
    A=np.maximum(0,Z)
    cache.append(Z)
    return A, cache

In [ ]:
def initialize_parameters(layer_dims):
    parameters = {}
    np.random.seed(3)
    for i in range(1, len(layer_dims)):
        parameters['W' + str(i)] = np.random.randn(layer_dims[i], layer_dims[i - 1]) * 0.01
        parameters['b' + str(i)] = np.zeros((layer_dims[i], 1))
    return parameters

In [ ]:
def linear_forward(A,W,b):
    Z=np.dot(W,A)+b
    cache=(A,W,b)
    return Z,cache

In [ ]:
def linear_activation_forward(A_prev,W,b,activation):
    linear_cache = (A_prev, W, b)
    Z = np.dot(W, A_prev) + b
    if activation == 'sigmoid':
        A, activation_cache = sigmoid(Z)
    elif activation == 'relu':
        A, activation_cache = relu(Z)
    cache = (linear_cache, activation_cache)
    return A, cache

In [ ]:
def L_model_forward(X,parameters):
    caches=[]
    A=X
    L=len(parameters)//2
    for i in range(1,L):
        A_prev=A
        A,cache=linear_activation_forward(A_prev,parameters['W'+str(i)],parameters['b'+str(i)],'relu')
        caches.append(cache)
    AL,cache=linear_activation_forward(A,parameters['W'+str(L)],parameters['b'+str(L)],'sigmoid')
    caches.append(cache)
    return AL,caches

In [ ]:
def compute_cost(AL,Y):
    m=Y.shape[1]
    cost=-np.sum(Y*np.log(AL)+(1-Y)*np.log(1-AL))/m
    cost=np.squeeze(cost)
    return cost

In [ ]:
def linear_backward(dZ,cache):
    A_prev,W,b=cache
    m=A_prev.shape[1]
    dW=np.dot(dZ,A_prev.T)/m
    db=np.sum(dZ,axis=1,keepdims=True)/m
    dA_prev=np.dot(W.T,dZ)
    return dA_prev,dW,db

In [ ]:
def sigmoid_backward(dA,cache):
    Z=cache[0]
    s=1/(1+np.exp(-Z))
    dZ=dA*s*(1-s)
    return dZ

In [ ]:
def relu_backward(dA,cache):
    Z=cache[0]
    dZ=np.array(dA,copy=True)
    dZ[Z<=0]=0
    return dZ

In [ ]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    if activation == 'relu':
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    elif activation == 'sigmoid':
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    return dA_prev, dW, db

In [ ]:
def L_model_backward(AL, Y, caches):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    current_cache = caches[L - 1]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, 'sigmoid')
    for l in reversed(range(L - 1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 2)], current_cache, 'relu')
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
    return grads

In [ ]:
def update_parameters(params, grads, learning_rate):
    parameters = params.copy()
    L = len(parameters) // 2
    for l in range(L):
        parameters["W" + str(l + 1)] = parameters["W" + str(l + 1)] - learning_rate * grads["dW" + str(l + 1)]
        parameters["b" + str(l + 1)] = parameters["b" + str(l + 1)] - learning_rate * grads["db" + str(l + 1)]
    return parameters

In [ ]:
def L_layer_model(X, Y, layers_dims, learning_rate = 0.075, num_iterations = 3000, print_cost=False):
    np.random.seed(1)
    costs = []
    parameters = initialize_parameters(layers_dims)
    for i in range(0, num_iterations):
        AL, caches = L_model_forward(X, parameters)
        cost = compute_cost(AL, Y)
        grads = L_model_backward(AL, Y, caches)
        parameters = update_parameters(parameters, grads, learning_rate)
        if print_cost and (i % 100 == 0 or i == num_iterations - 1):
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0:
            costs.append(cost)
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    return parameters, costs

In [ ]:
parameters, costs = L_layer_model(image, label, layers_dims, num_iterations = 2500, print_cost = True)

In [ ]:
def predict(train_x,train_y,parameters):
    m=train_x.shape[1]
    n=len(parameters)//2
    p=np.zeros((1,m))
    probas,caches=L_model_forward(train_x,parameters)
    for i in range(0,probas.shape[1]):
        if probas[0,i]>0.5:
            p[0,i]=1
        else:
            p[0,i]=0
    print("Accuracy: "  + str(np.sum((p == train_y)/m)))
    return p

In [ ]:
pred_train = predict(image, label, parameters)

In [ ]:
for x in ds_test.take(1):
    image, label = x
    plt.figure(figsize=(1, 1))
    plt.imshow(image.numpy()[3])  # Convert tensor to NumPy array.
    plt.title(f'Label: {label.numpy()[3]}')
    plt.axis('off')  # Suppress axis display for clarity.
    plt.show()
    label1 = label.numpy().reshape(-1, 1)
    image1 = image.numpy().reshape(image.shape[0], -1).T

In [ ]:
pred_test = predict(image1, label1, parameters)

In [ ]:
## START CODE HERE ##
from PIL import Image
my_image = "my_image.jpg" # change this to the name of your image file 
my_label_y = [1] # the true class of your image (1 -> cat, 0 -> non-cat)
## END CODE HERE ##
num_px=32
fname = "images/" + my_image
image = np.array(Image.open(fname).resize((num_px, num_px)))
image = image / 255.
image = image.reshape((1, num_px * num_px * 3)).T
classes = ["cat", "non-cat"]
my_predicted_image = predict(image, my_label_y, parameters)

print ("y = " + str(np.squeeze(my_predicted_image)) + ", your L-layer model predicts a \"" + classes[int(np.squeeze(my_predicted_image))] +  "\" picture.")